<a href="https://colab.research.google.com/github/alexeiveselov92/Yandex-Data-Analysis/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color='blue'> Привет. Извини за долгую проверку. Ниже мои комментарии выделены синим. </font>

# Магазин продуктов питания. A/B тест - использование новых шрифтов.


## Описание проекта
Вы работаете в стартапе, который продаёт продукты питания. Нужно разобраться, как ведут себя пользователи вашего мобильного приложения.

Изучите воронку продаж. Узнайте, как пользователи доходят до покупки. Сколько пользователей доходит до покупки, а сколько — «застревает» на предыдущих шагах? На каких именно?

После этого исследуйте результаты A/B-эксперимента. Дизайнеры захотели поменять шрифты во всём приложении, а менеджеры испугались, что пользователям будет непривычно. Договорились принять решение по результатам A/B-теста. Для него пользователей разбили на 3 группы: 2 контрольные со старыми шрифтами и одну экспериментальную — с новыми. Выясните, какой шрифт лучше.

В случае общей аналитики и A/B-эксперимента работайте с одними и теми же данными. В реальных проектах всегда идут эксперименты. Аналитики исследуют качество работы приложения по общим данным, не учитывая принадлежность пользователей к экспериментам.

<a id='content'></a>
## Оглавление

### Подготовка
* [Загрузка библиотек](#download_libraries)
* [Загрузка датасета и проверка его на наличие дубликатов, пропусков и соответствие типам данных, а также правильность выгрузки](#download_dataset)
* [Замена названий столбцов, добавление необходимых столбцов для удобной работы](#preparation)
   


### Изучим и проверим данные

* [Сколько всего событий в логе? Сколько всего пользователей в логе? Сколько в среднем событий приходится на пользователя?](#how_many_events)
* [Данными за какой период обладаем? Можем ли быть уверенными, что у нас одинаково полные данные за весь период?](#period)

### Изучим воронку событий

* [Посмотрим, какие события есть в логах, как часто они встречаются. Отсортируем события по частоте.](#funnel)
* [Посчитаем, сколько пользователей совершали каждое из этих событий. Отсортируйте события по числу пользователей. Посчитаем долю пользователей, которые хоть раз совершали событие.](#funnel)
* [Предположим, в каком порядке происходят события. Все ли они выстраиваются в последовательную цепочку? Их не нужно учитывать при расчёте воронки.](#funnel)
* [По воронке событий посчитайте, какая доля пользователей проходит на следующий шаг воронки (от числа пользователей на предыдущем). То есть для последовательности событий A → B → C, посчитайте отношение числа пользователей с событием B к количеству пользователей с событием A.](#funnel)
* [На каком шаге теряете больше всего пользователей?](#funnel)
* [Какая доля пользователей доходит от первого события до оплаты?](#funnel)

### Изучим результаты эксперимента
* [Сколько пользователей в каждой экспериментальной группе?](#results)
* [Есть 2 контрольные группы для А/А-эксперимента, чтобы проверить корректность всех механизмов и расчётов. Проверьте, находят ли статистические критерии разницу между выборками 246 и 247.](#results)
* [Посчитайте долю пользователей, совершивших это событие. Проверьте, будет ли отличие между группами статистически достоверным. Проделайте то же самое для всех других событий (удобно обернуть проверку в отдельную функцию). Можно ли сказать, что разбиение на группы работает корректно?](#results)
* [Аналогично поступите с группой с изменённым шрифтом. Сравните результаты с каждой из контрольных групп в отдельности по каждому событию. Сравните результаты с объединённой контрольной группой. Какие выводы из эксперимента можно сделать?](#results)
* [Какой уровень значимости вы выбрали при проверке статистических гипотез выше? Посчитайте, сколько проверок статистических гипотез вы сделали. При уровне значимости 0.1 каждый десятый раз можно получать ложный результат. Какой уровень значимости стоит применить? Если вы хотите изменить его, проделайте предыдущие пункты и проверьте свои выводы.](#results)

## Выводы

* [Выводы по проекту](#summary)

# Подготовка
<a id='download_libraries'></a>
### Загрузим библиотеки
[*Назад к оглавлению*](#content)

In [0]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly import graph_objects as go
import copy
import math as mth
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


<a id='download_dataset'></a>
### Загрузим датасет и проверим его на наличие дубликатов, пропусков и соответствие типам данных, а также правильность выгрузки
[*Назад к оглавлению*](#content)

In [0]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/datasets/logs_exp.csv', sep='\t')
print('Дубликатов обнаружено: {}\n'.format(df.duplicated().sum()))
df = df.drop_duplicates().reset_index(drop = True)
df.info()
print('\nДубликаты удалены')

print('\nПроверим категорийную переменную:\n{}\n\nВсе впорядке, идём дальше'.format(df['EventName'].value_counts()))
df.head(5)

Дубликатов обнаружено: 413

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243713 entries, 0 to 243712
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   EventName       243713 non-null  object
 1   DeviceIDHash    243713 non-null  int64 
 2   EventTimestamp  243713 non-null  int64 
 3   ExpId           243713 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 7.4+ MB

Дубликаты удалены

Проверим категорийную переменную:
MainScreenAppear           119101
OffersScreenAppear          46808
CartScreenAppear            42668
PaymentScreenSuccessful     34118
Tutorial                     1018
Name: EventName, dtype: int64

Все впорядке, идём дальше


,EventName,DeviceIDHash,EventTimestamp,ExpId
0,MainScreenAppear,4575588528974610257,1564029816,246
1,MainScreenAppear,7416695313311560658,1564053102,246
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248


<a id='preparation'></a>
### Замена названий столбцов, добавление необходимых столбцов для удобной работы
[*Назад к оглавлению*](#content)

In [0]:
df.columns = ['event', 'user_id', 'ts', 'group_id']
df['datetime'] = pd.to_datetime(df['ts'], unit='s')
df['date'] = pd.to_datetime(df['datetime'].dt.date)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243713 entries, 0 to 243712
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   event     243713 non-null  object        
 1   user_id   243713 non-null  int64         
 2   ts        243713 non-null  int64         
 3   group_id  243713 non-null  int64         
 4   datetime  243713 non-null  datetime64[ns]
 5   date      243713 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(3), object(1)
memory usage: 11.2+ MB


In [0]:
print('Подготовка завершена, идём дальше')
df.head(5)

Подготовка завершена, идём дальше


,event,user_id,ts,group_id,datetime,date
0,MainScreenAppear,4575588528974610257,1564029816,246,2019-07-25 04:43:36,2019-07-25
1,MainScreenAppear,7416695313311560658,1564053102,246,2019-07-25 11:11:42,2019-07-25
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248,2019-07-25 11:28:47,2019-07-25
3,CartScreenAppear,3518123091307005509,1564054127,248,2019-07-25 11:28:47,2019-07-25
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248,2019-07-25 11:48:42,2019-07-25


# Изучим и проверим данные
<a id='how_many_events'></a>
### Сколько всего событий в логе? Сколько всего пользователей в логе? Сколько в среднем событий приходится на пользователя?
[*Назад к оглавлению*](#content)

In [0]:
print('Событий в логе: {}'.format(len(df)))
print('Пользователей в логе: {}'.format(len(df['user_id'].unique())))
print('В среднем событий на одного пользователя: {:.0f}'.format(stats.mode(df.groupby('user_id').agg({'event':'count'}))[0][0][0]))

Событий в логе: 243713
Пользователей в логе: 7551
В среднем событий на одного пользователя: 5


<font color='blue'> Хорошо, верно. </font>

<a id='period'></a>
### Данными за какой период обладаем?   Можем ли быть уверенными, что у нас одинаково полные данные за весь период?   
[*Назад к оглавлению*](#content)

**Выполним по пунктам:**
* Найдем максимальную и минимальную дату.
* Построим гистограмму по дате и времени.
* Так как технически в логи новых дней по некоторым пользователям могут «доезжать» события из прошлого — это может «перекашивать данные», определим, с какого момента данные полные и отбросим более старые.
* Данными за какой период времени мы располагаете на теперь?
* Много ли событий и пользователей мы потеряли, отбросив старые данные?
* Проверим, что у нас есть пользователи из всех трёх экспериментальных групп.

In [0]:
print('Минимальная дата: {}'.format(df['date'].min()))
print('Максимальная дата: {}'.format(df['date'].max()))
print('Период составляет: {}'.format(df['datetime'].max()-df['datetime'].min()))

event_date_grouped = df.groupby('date', as_index=False).agg({'datetime':'count'}).reset_index()
event_date_grouped['date_text'] = event_date_grouped['date'].dt.strftime('%d.%m')
# df.head(5)
fig = px.bar(event_date_grouped, x = 'date', y='datetime', text='date_text', width=750, height=400)
fig.update_layout(
    title={
        'text': "Распределение количества событий по дням",
        'xanchor': 'center',
        'x':0.5
    },
    yaxis_title="Количество событий",
    xaxis_title="Дата")
fig.update_traces(textposition='outside')
fig.show()
print('Полные данные у нас есть только за период с 1 по 7 августа включительно.'
      '\nОтбросим периоды, по которым данных недостаточно.')

Минимальная дата: 2019-07-25 00:00:00
Максимальная дата: 2019-08-07 00:00:00
Период составляет: 13 days 16:31:41


Полные данные у нас есть только за период с 1 по 7 августа включительно.
Отбросим периоды, по которым данных недостаточно.


In [0]:
events_before_filtration = len(df)
users_before_filtration = len(df.groupby('user_id')['date'].count())
print('Событий до фильтрации: {}\nПользователей до фильтрации: {}'\
     .format(events_before_filtration, users_before_filtration))
df = df[(df['date']>='2019-08-01')]
events_after_filtration = len(df)
users_after_filtration = len(df.groupby('user_id')['date'].count())
print('\nСобытий после фильтрации: {} (-{:.2%})'
      '\nПользователей после фильтрации: {} (-{:.2%})\n'\
     .format(events_after_filtration,
             ((events_before_filtration-events_after_filtration)/events_before_filtration),
             users_after_filtration,
             ((users_before_filtration-users_after_filtration)/users_before_filtration)))
df_grouped_by_groups = df.groupby('group_id', as_index=False).agg({'user_id':['nunique', 'count']})
df_grouped_by_groups.columns = ['group_id', 'users', 'events']
print('Кол-во событий и пользователей после фильтрации:\n{}'.format(df_grouped_by_groups))
print('\nПосле фильтрации присутствуют события из всез трёх групп. Всё хорошо, идём дальше.')

Событий до фильтрации: 243713
Пользователей до фильтрации: 7551

Событий после фильтрации: 240887 (-1.16%)
Пользователей после фильтрации: 7534 (-0.23%)

Кол-во событий и пользователей после фильтрации:
   group_id  users  events
0       246   2484   79302
1       247   2513   77022
2       248   2537   84563

После фильтрации присутствуют события из всез трёх групп. Всё хорошо, идём дальше.


<font color='blue'>Ок, процент вполне приемлимый, группы сопоставимы, можно переходить к анализу. </font>

<a id='funnel'></a>
# Изучим воронку событий
[*Назад к оглавлению*](#content)

**Выполним по пунктам:**
* Посмотрим, какие события есть в логах, как часто они встречаются. Отсортируем события по частоте.
* Посчитаем, сколько пользователей совершали каждое из этих событий. Отсортируем события по числу пользователей.
* Посчитаем долю пользователей, которые хоть раз совершали событие. 
* Предположим, в каком порядке происходят события. Все ли они выстраиваются в последовательную цепочку? Их не нужно учитывать при расчёте воронки. 
* По воронке событий посчитаем, какая доля пользователей проходит на следующий шаг воронки (от числа пользователей на предыдущем). То есть для последовательности событий A → B → C, посчитаем отношение числа пользователей с событием B к количеству пользователей с событием A. На каком шаге теряем больше всего пользователей? Какая доля пользователей доходит от первого события до оплаты?


In [0]:
df_grouped_by_events = df.groupby('event', as_index=False).agg({'user_id':['count', 'nunique']})
df_grouped_by_events.columns = ['event', 'events', 'users']
df_grouped_by_events = df_grouped_by_events.sort_values(by='events', ascending=False)
print('События, отсортированные по частоте и количеству пользователей:')
df_grouped_by_events

События, отсортированные по частоте и количеству пользователей:


,event,events,users
1,MainScreenAppear,117328,7419
2,OffersScreenAppear,46333,4593
0,CartScreenAppear,42303,3734
3,PaymentScreenSuccessful,33918,3539
4,Tutorial,1005,840


In [0]:
print('\nКак мы видим из полученной таблицы, одно событие Tutorial не вписывается в нашу воронку.'
     '\nСудя по-всему это событие не обязательно для каждого пользователя и идёт первым или вторым по порядку исходя из логики.'
     '\nНе будем его учитывать в воронке.')

df_grouped_by_events['part_of_users'] = (df_grouped_by_events['users']/df_grouped_by_events['users'].max()).round(2)
df_grouped_by_events['part_of_last_event'] = 1 + df_grouped_by_events['users'].pct_change()
events_funnel = df_grouped_by_events[df_grouped_by_events['event']!='Tutorial']
print(events_funnel)
print('\nБольше всего пользователей теряем на первом шаге воронки MainScreenAppear - OffersScreenApppear.')
print('До оплаты же доходит около 48% пользователей.')
print('Но все ли из них прошли через каждый шаг?')
print('\nКакие события есть:')
print('''MainScreenAppear - главный экран;
Tutorial - экран с обучением;
OffersScreenAppear - экран с товаром;
CartScreenAppear - экран корзины;
PaymentScreenSuccessful - экран успешной оплаты заказа.'''
)

events_per_user = df.pivot_table(index='user_id', values='group_id', columns='event', aggfunc='count')\
[['Tutorial', 'MainScreenAppear', 'OffersScreenAppear', 'CartScreenAppear', 'PaymentScreenSuccessful']].reset_index()

#список купивших пользователей
purchased_users_list = events_per_user[~events_per_user['PaymentScreenSuccessful'].isnull()]['user_id']
#список не купивших пользователей
not_purchased_users_list = events_per_user[events_per_user['PaymentScreenSuccessful'].isnull()]['user_id']

events_by_all_users = df.groupby('event', as_index=False).agg({'user_id':'nunique'})
events_by_all_users['percent_of_all'] = events_by_all_users['user_id'] / len(df.groupby('user_id'))
events_by_all_users.columns = ['event', 'all_users', 'percent_of_all']

events_by_purchased_users = df.query('user_id in @purchased_users_list').groupby('event', as_index=False).agg({'user_id':'nunique'})
events_by_purchased_users['percent_of_purchased'] = events_by_purchased_users['user_id'] / len(df.query('user_id in @purchased_users_list').groupby('user_id'))
events_by_purchased_users.columns = ['event', 'purchased_users', 'percent_of_purchased']

events_by_not_purchased_users = df.query('user_id not in @purchased_users_list').groupby('event', as_index=False).agg({'user_id':'nunique'})
events_by_not_purchased_users['percent_of_not_purchased'] = events_by_not_purchased_users['user_id'] / len(df.query('user_id not in @purchased_users_list').groupby('user_id'))
events_by_not_purchased_users.columns = ['event', 'not_purchased_users', 'percent_of_not_purchased']

events_by_users = events_by_all_users.merge(events_by_purchased_users, on='event')
events_by_users = events_by_users.merge(events_by_not_purchased_users, how='left', on='event')
events_by_users = events_by_users.sort_values(by='all_users', ascending = False)
print('\nДоля купивших пользователей, прошедших события:\n{}\n\nКак мы видим, не каждый купивший клиент прошел через все этапы'.format(events_by_users[['event', 'purchased_users', 'percent_of_purchased']]))


Как мы видим из полученной таблицы, одно событие Tutorial не вписывается в нашу воронку.
Судя по-всему это событие не обязательно для каждого пользователя и идёт первым или вторым по порядку исходя из логики.
Не будем его учитывать в воронке.
                     event  events  users  part_of_users  part_of_last_event
1         MainScreenAppear  117328   7419           1.00                 NaN
2       OffersScreenAppear   46333   4593           0.62            0.619086
0         CartScreenAppear   42303   3734           0.50            0.812976
3  PaymentScreenSuccessful   33918   3539           0.48            0.947777

Больше всего пользователей теряем на первом шаге воронки MainScreenAppear - OffersScreenApppear.
До оплаты же доходит около 48% пользователей.
Но все ли из них прошли через каждый шаг?

Какие события есть:
MainScreenAppear - главный экран;
Tutorial - экран с обучением;
OffersScreenAppear - экран с товаром;
CartScreenAppear - экран корзины;
PaymentScreenSuccessful - эк

In [0]:
purchased = copy.copy(events_by_purchased_users[:-1])
purchased.columns = ['event', 'users', 'percent']
purchased = purchased.loc[[1, 2, 0, 3]]
purchased['event'] = ["Посетителей","Открыли страницу товара", "Перешли в корзину", "Оплатили"]
purchased['status'] = 'purchased'

not_purchased = copy.copy(events_by_not_purchased_users[:-1])
not_purchased = not_purchased.loc[[1, 2, 0]]
not_purchased.columns = ['event', 'users', 'percent']
not_purchased.loc[3] = ['PaymentScreenSuccessful', 0, 0]
not_purchased['event'] = ["Посетителей","Открыли страницу товара", "Перешли в корзину", "Оплатили"]
not_purchased['status'] = 'not purchased'

fig = go.Figure()
fig.add_trace(go.Funnel(
    name = 'Купившие посетители',
    y = purchased['event'],
    x = purchased['users'],
    textinfo = "value+percent previous"))
fig.add_trace(go.Funnel(
    name = 'Не купившие посетители',
    y = not_purchased['event'],
    x = not_purchased['users'],
    textinfo = "value+percent previous"))
fig.update_layout(title={'text': "Воронка событий", 'xanchor': 'center', 'x':0.5, 'y':0.87})
fig.show()

<font color='blue'> Воронка рассчитана верно. Надо было посмотреть еще рассчитать и визуализировать долю пользователей на каждом этапе.Также надо было построить воронку без разделения, а только потом, если это интресно, с разделением на купивших и не купивших пользователей </font>

<font color='#2b6ca3'>Не совсем понял - долю пользователей на каждом этапе от общего числа или от прошлого этапа? Они уже были посчитаны выше, вот снизу еще раз вставил таблицу, по ней мы и определили, что мы больше всего теряем на данном этапе пользователей - MainScreenAppear - OffersScreenAppear. 

Вот ниже сделал общую воронку с указанием процентов от предыдущего этапа.</font>

In [0]:
print(events_funnel)

fig = go.Figure()
fig.add_trace(go.Funnel(
    y = events_funnel['event'],
    x = events_funnel['users'],
    textinfo = "value+percent previous"))
fig.update_layout(title={'text': "Воронка событий", 'xanchor': 'center', 'x':0.5, 'y':0.87})
fig.show()

                     event  events  users  part_of_users  part_of_last_event
1         MainScreenAppear  117328   7419           1.00                 NaN
2       OffersScreenAppear   46333   4593           0.62            0.619086
0         CartScreenAppear   42303   3734           0.50            0.812976
3  PaymentScreenSuccessful   33918   3539           0.48            0.947777


<font color='red'> Ок.  Имелась ввиду доля пользователей от общего числа. Доля от предыдущего этапа - это воронка. Дялее цитата из брифа  </font>

       Посчитайте, сколько пользователей совершали каждое из этих событий. 
       
       Отсортируйте события по числу пользователей. 
       
       Посчитайте долю пользователей, которые хоть раз совершали событие

<a id='results'></a>
# Изучим результаты эксперимента
[*Назад к оглавлению*](#content)

**Выполним по пунктам:**
* Сколько пользователей в каждой экспериментальной группе?
* Есть 2 контрольные группы для А/А-эксперимента, чтобы проверить корректность всех механизмов и расчётов. Проверьте, находят ли статистические критерии разницу между выборками 246 и 247.
* Посчитайте долю пользователей, совершивших это событие. Проверьте, будет ли отличие между группами статистически достоверным. Проделайте то же самое для всех других событий (удобно обернуть проверку в отдельную функцию). Можно ли сказать, что разбиение на группы работает корректно?
* Аналогично поступите с группой с изменённым шрифтом. Сравните результаты с каждой из контрольных групп в отдельности по каждому событию. Сравните результаты с объединённой контрольной группой. Какие выводы из эксперимента можно сделать?
* Какой уровень значимости вы выбрали при проверке статистических гипотез выше? Посчитайте, сколько проверок статистических гипотез вы сделали. При уровне значимости 0.1 каждый десятый раз можно получать ложный результат. Какой уровень значимости стоит применить? Если вы хотите изменить его, проделайте предыдущие пункты и проверьте свои выводы.

In [0]:
print('Пользователей по группам:\n{}'.format(df_grouped_by_groups[['group_id', 'users']]))

Пользователей по группам:
   group_id  users
0       246   2484
1       247   2513
2       248   2537


In [0]:
pd.options.display.expand_frame_repr = False
event_parts_per_groups = df.pivot_table(index='event', values='user_id', columns='group_id', aggfunc='nunique').reset_index()
event_parts_per_groups = event_parts_per_groups[:-1]
users_by_group_246 = event_parts_per_groups[246].sum()
users_by_group_247 = event_parts_per_groups[247].sum()
users_by_group_248 = event_parts_per_groups[248].sum()
event_parts_per_groups['246_247'] = event_parts_per_groups[246] + event_parts_per_groups[247]
users_by_group_246_247 = event_parts_per_groups['246_247'].sum()
event_parts_per_groups['246_part'] = event_parts_per_groups[246] / users_by_group_246
event_parts_per_groups['247_part'] = event_parts_per_groups[247] / users_by_group_247
event_parts_per_groups['248_part'] = event_parts_per_groups[248] / users_by_group_248
event_parts_per_groups = event_parts_per_groups.set_index('event')
event_parts_per_groups['246_247_part'] = event_parts_per_groups['246_247'] / users_by_group_246_247
print(event_parts_per_groups)

print('\nПроверим нулевые гипотезы о том, что доли событий в группах 246 и 247 равны по-очередно:\n')
alpha =0.05

def Z_test(index, group_1, group_2, part_1, part_2, number_1, number_2):
    print('{}:'.format(index))
    p1 = event_parts_per_groups[part_1][index]
    p2 = event_parts_per_groups[part_2][index]
    p = (event_parts_per_groups[group_1][index]+event_parts_per_groups[group_2][index])/\
    (users_by_group_246 + users_by_group_247)
    n1 = number_1
    n2 = number_2
    z_value = (p1-p2)/mth.sqrt(p * (1-p) * (1/n1 + 1/n2))
    distr = stats.norm(0, 1)
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    print('p-значение: ', p_value)
    if p_value < alpha:
        print("Отвергаем нулевую гипотезу: между долями есть значимая разница")
    else:
        print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными")

Z_test('CartScreenAppear', 246, 247, '246_part', '247_part', users_by_group_246, users_by_group_247)
Z_test('MainScreenAppear', 246, 247, '246_part', '247_part', users_by_group_246, users_by_group_247)
Z_test('OffersScreenAppear', 246, 247, '246_part', '247_part', users_by_group_246, users_by_group_247)
Z_test('PaymentScreenSuccessful', 246, 247, '246_part', '247_part', users_by_group_246, users_by_group_247)
print('Разницы между группами 246 и 247 не обнаружено')

if df.groupby('user_id').agg({'group_id':'nunique'}).sum()[0] == len(df.groupby('user_id').agg({'group_id':'nunique'})):
    print('\nПользователи разбиты на группы корректно, одних и тех же пользователей в нескольких'
         '\nгруппах одновременно не наблюдается.')

group_id                  246   247   248  246_247  246_part  247_part  248_part  246_247_part
event                                                                                         
CartScreenAppear         1266  1238  1230     2504  0.196036  0.193680  0.191142      0.194864
MainScreenAppear         2450  2476  2493     4926  0.379374  0.387359  0.387413      0.383346
OffersScreenAppear       1542  1520  1531     3062  0.238774  0.237797  0.237918      0.238288
PaymentScreenSuccessful  1200  1158  1181     2358  0.185816  0.181164  0.183528      0.183502

Проверим нулевые гипотезы о том, что доли событий в группах 246 и 247 равны по-очередно:

CartScreenAppear:
p-значение:  0.7359876852552283
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
MainScreenAppear:
p-значение:  0.35195027387494493
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
OffersScreenAppear:
p-значение:  0.8966517949162409
Не получилось отвергнуть нулев

<font color='blue'> Функция для проверки написана верно. </font>

<font color='blue'> Хорошо, АА тест пройден, можно переходить к АВ тесту. </font>

In [0]:
print('Сравним группу с измененным шрифтом 248 с контрольной группой 246')

print('\nПроверим нулевые гипотезы о том, что доли событий в группах 246 и 248 равны по-очередно:')
alpha =0.05
Z_test('CartScreenAppear', 246, 248, '246_part', '248_part', users_by_group_246, users_by_group_248)
Z_test('MainScreenAppear', 246, 248, '246_part', '248_part', users_by_group_246, users_by_group_248)
Z_test('OffersScreenAppear', 246, 248, '246_part', '248_part', users_by_group_246, users_by_group_248)
Z_test('PaymentScreenSuccessful', 246, 248, '246_part', '248_part', users_by_group_246, users_by_group_248)
print('\nРазницы между группами 246 и 248 не обнаружено\n\n')

print('Сравним группу с измененным шрифтом 248 с контрольной группой 247')

print('\nПроверим нулевые гипотезы о том, что доли событий в группах 247 и 248 равны по-очередно:')
alpha =0.05
Z_test('CartScreenAppear', 247, 248, '247_part', '248_part', users_by_group_247, users_by_group_248)
Z_test('MainScreenAppear', 247, 248, '247_part', '248_part', users_by_group_247, users_by_group_248)
Z_test('OffersScreenAppear', 247, 248, '247_part', '248_part', users_by_group_247, users_by_group_248)
Z_test('PaymentScreenSuccessful', 247, 248, '247_part', '248_part', users_by_group_247, users_by_group_248)
print('\nРазницы между группами 247 и 248 не обнаружено\n\n')

print('Сравним группу с измененным шрифтом 248 с объединенной контрольной группой 246-247')

print('\nПроверим нулевые гипотезы о том, что доли событий в группах 246-247 и 248 равны по-очередно:')
alpha =0.05
Z_test('CartScreenAppear', '246_247', 248, '246_247_part', '248_part', users_by_group_246_247, users_by_group_248)
Z_test('MainScreenAppear', '246_247', 248, '246_247_part', '248_part', users_by_group_246_247, users_by_group_248)
Z_test('OffersScreenAppear', '246_247', 248, '246_247_part', '248_part', users_by_group_246_247, users_by_group_248)
Z_test('PaymentScreenSuccessful', '246_247', 248, '246_247_part', '248_part', users_by_group_246_247, users_by_group_248)
print('\nРазницы между объединенной группой 246-247 и 248 не обнаружено\n\n')

print('\nВ рамках множественного сравнения групп мы использовали критический уровень значимости 0.05.'
     '\nПри множественном тестировании иногда используют поправки критического уровня значимости (уменьшение его)'
     '\nдля снижения вероятности ошибки первого рода. В нашем случае p-value в каждом тесте достаточно высок,'
     '\nдля того, чтобы введение поправок критического уровня значимости было бессмысленным. Это не изменит результаты.'
     '\nБолее того никто в курсе так и не удосужился нам объянсить толково смысл этих поправок'
     ' и ответить \nна мои вопросы на этот счёт. Так что, я вообще не понимаю зачем они нам нужны до сих пор. Ересь полная.')

Сравним группу с измененным шрифтом 248 с контрольной группой 246

Проверим нулевые гипотезы о том, что доли событий в группах 246 и 248 равны по-очередно:
CartScreenAppear:
p-значение:  0.4825014804387422
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
MainScreenAppear:
p-значение:  0.3482416320635866
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
OffersScreenAppear:
p-значение:  0.9092955840456791
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
PaymentScreenSuccessful:
p-значение:  0.7380812690877536
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Разницы между группами 246 и 248 не обнаружено


Сравним группу с измененным шрифтом 248 с контрольной группой 247

Проверим нулевые гипотезы о том, что доли событий в группах 247 и 248 равны по-очередно:
CartScreenAppear:
p-значение:  0.7152884920301557
Не получилось отвергнуть нулевую гипотезу, нет оснований считать до

<font color='blue'> Хорошо. Да гипотезы проверены правильно. </font>

<font color='blue'> Выводы в целом верны, хотя все же надо рассчитать alpha с учетом поправок и только потом написать, что оно меньше 0.05 и меньше всех полученых p_value. </font>

<font color='blue'> Продробнее прочитать про поправки можно, например, тут. </font>

https://r-analytics.blogspot.com/2013/10/blog-post_13.html

<font color='#2b6ca3'> Прочитал, спасибо! Я не понимаю применять их надо ко всем 16 тестам в рамках проекта или же тольво внутри групп, где проводятся по 4 тестирования. Ответов на этот вопрос нам не давали. 
Более того я сам задавал этот вопрос и мне обещали дать ответ от разработчиков, но увы. (во время прохождения теории этот момент я как раз и не понял).

При увеличении количества абсолютно любых тестов, вероятность ошибки первого рода хотябы в одном из них будет увеличиваться - независимо от того, в рамках одного проекта тесты или же в рамках одной ГС или же в рамках одного дня, в который в научном институте было проведено 120 A/B тестов по разным темам. В случае с институтом за 1 этот день мы должны были бы совершить 6 ошибок первого рода в среднем.
    
Так вот учитывая этот факт непонятно когда надо применять эти поправки, а когда не надо. Почему их надо применять в группах но не применять вне групп? Что считать группой - одно исследование, одну группу из ГС или все группы из одной ГС? 

Более того я так же задавал разработчикам вопрос - уменьшая крит. уровень занчимости (alpha), мы увеличиваем вероятность ошибки второго рода. Но почему никто об этом не упоминает при множественном сравнении? Мы беспокоимся об ошибке первого рода, уменьшая её вероятность, увеличиваем вероятность ошибки второго рода. Отклонить крутое изменение не так страшно, как сделать бесполезное изменение, которое ничего не изменит? Ересь полная получается.  </font>

<font color='red'> Очень много вопросов связанго с процедурой проведения эксперимента и ответ на них будет разный при различных постановках эксперимента. Здесь речь идет не о том сколько дней мы проводим эксперимент и сколько еще идет эксперисентов в нашем офисе. Речь об одном конкретном проекте. Мы собрали для него несколько групп и хотим проверить сразу несколько гипотез, чтоб не собирать много раз данные - это затратно. Но тогда есть шанс, что одна проверка влияет на другую и именно это должна учитывать поправка </font>


<font color='red'> Ты гне совсем точно формулируешь. Ошибка первого рода - принятие негативных изменений. Ошибка второго рода - отклонение позитивных. Да, для нас важнее исключить ошибку первого рода. Т.е принятие негативных изменений недопустимо. При этом возможно мы пропустим полезные изменения - за все надо платить. Негативные изменения не допустимы, т.к. они потратят наше время во вред и отпугнут пользователей. После этого можно сколько угодно делать положительные изменения, но пользователи уже ушли...  </font>


<font color='red'> Поправки в даннном случае рассчтываются так  </font>




<a id='summary'></a>
# Выводы
[*Назад к оглавлению*](#content)

На этапе подготовки данных мы выявили несущественное количество дублей строк и удалили их.

Событий в логе: 240887
Пользователей в логе: 7534
В среднем событий на одного пользователя: 6

Данные в логе были за период в 2 недели, но мы выяснили, что полноценные данные были лишь за неделю, поэтому неполноценные данные мы отсекли и взяли период с 1 по 7 августа. Количество данных уменьшилось несущественно.

В нашу воронку из лога мы взяли 4 события из 5, так как событие Tutorial необязательно для пользователей и для нас не показательно. 

В нашей воронке по количеству события распологаются в логичном порядке:
* MainScreenAppear - 117328 событий, 7419 пользователей, доля 1.00                
* OffersScreenAppear - 46333 событий, 4593 пользователей, доля 0.62           
* CartScreenAppear - 42303 событий, 3734 пользователей, доля 0.50            
* PaymentScreenSuccessful - 33918 событий, 3539 пользователей, доля 0.48    

Больше всего пользователей теряем на первом шаге воронки MainScreenAppear - OffersScreenApppear.
До оплаты же доходит около 48% пользователей.

Дополнительно мы выяснили, что купившие пользователи не обязательно проходили все предыдущие события до попадания на экран покупки, но доля таких пользователей не существенна, хотя и имеет место быть. Это может быть предметом обсуждения с коллегами.

Критический уровень значимости перед проведением тестирований мы взяли 0.05.
Перед A/B тестирование мы провели A/A тестирование групп 246 и 247, выяснилось что всё впорядке и распределение на группы было корректным.

Мы провели A/B тестирование и не заметили статистически значимых отличий между контрольными группами и экспериментальной. 

Вывод - изменение шрифта не привело ни к каким статистически значимым отличиям.

Проводя множественное сравнение при A/A и A/B тестированиях, мы не использовали никакие поправки критического уровня значимости, так как в этом не было никакого смысла - при уменьшении критического уровня значимости с помощью имеющихся поправок результаты тестирования не изменились бы, это видно из полученных p-value (минимальное p-value - 0.34).

От себя:

Нам УЖАСНО преподали тему Бонферони и других поправок. Толковых объяснений зачем их применять не было, кроме того, что чем больше экспериментов, то вероятность ошибки первого рода растет с их количеством. (это и ежу понятно, ток нафига это необходимо только при множественном сравнении, почему нельзя применять тогда при разных A/B тестах на разных ГС? Мне обещали уже очень давно дать ответ от разработчиков, но так и не дали.)

<font color='blue'> Ты проделал хорошую работу, но нужно немного доработать, а именно: отобразить джолю пользователей для каждого события, построить общую вороо. </font>

<font color='#2b6ca3'>Отобразил долю пользователей в воронке, в таблице эти значения уже были.</font>